# PUC Minas EAD - Trabalho de conclusão de curso

Trabalho de conclusão de curso de Guilherme Fernando Angélico para o título de especialista em Inteligência Artificial e Machine Learning. 28/02/2022

In [ ]:
from platform import python_version
print('Versão da linguagem python utilizada para a execução desse notebook:', python_version())

## Detectando transações fraudulentas de cartão de crédito com Inteligência Artificial - Uma abordagem não supervisionada

!['Análise automatica de transações fraldulentas de cartão de crédito'](./images/analise-manual-de-risco.png)

## Definição do problema

Os sistemas comerciais sempre estão em constante evolução, e transacionar uma compra em um e-commerce é uma operação cada vez mais comum; E na mesma velocidade em que as transações ocorrem sempre há uma possibilidade de que essa transação seja fraudulenta, ou seja, não é o dententor do cartão efetuando a compra - uma pessoa maliciosa pode ter capturado esses dados e se passar por essa pessoa.

Nesse projeto, vamos criar um sistema capaz de analisar as transações financeiras e criar um modelo capaz de prever uma classe para cada transação indicando se ela é uma Transação  **Válida** ou um **Fraude**.

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from matplotlib import rcParams
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, f1_score ,precision_score, recall_score, accuracy_score, precision_recall_curve, classification_report

sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8

In [ ]:
%reload_ext watermark
%watermark -a "Análise de fraudes em transações com cartões de crédito" --iversions

## Dataset

Para modelar nosso sistema de análise de transações fraudulentas vamos utilizar um dataset público disponível no Kaggle:

https://www.kaggle.com/mlg-ulb/creditcardfraud

O dataset possui um total de 31 colunas. Sendo 28 colunas com valores numérico/decimal, obtidos através do processo de PCA - afim de proteger as identidades e recursos confidencias, e outras 3 colunas, sendo: Número de segundos entre a primeira transação do dataset e a atual, O valor da transação e a Classe.

In [ ]:
RANDOM_SEED = 42
LABELS = ["Normal", "Fraude"]

In [ ]:
df = pd.read_csv('./data/creditcard.zip', sep=',', compression='zip')
df.head()

## Análise do dataset

Vamos analisar o dataset e identicar as caracteristicas das transações desse dataset

In [ ]:
qtd_class = pd.value_counts(df['Class'], sort = True)
plt.title("Distribuição")
ax = sns.barplot(x=qtd_class.keys(), y=qtd_class.values)

for p in ax.patches:
    percent = '{:.1f}%'.format(100 * p.get_height()/len(df))
    x = p.get_x() + p.get_width() / 2 - 0.05
    y = p.get_y() + p.get_height()
    ax.annotate(percent, (x, y), size=12)

ax.set_xticks(range(len(LABELS)), LABELS)
ax.set_xlabel("Classe")
ax.set_ylabel("Quantidade")
plt.show()

Vamos criar uma coluna de data/tempo para poder colocar nososs registros dentro de uma time series

In [ ]:
data_base = dt.datetime(2013, 9, 1, 8, 0, 0)
data_base

In [ ]:
df['Data'] = df['Time'].apply(lambda x: data_base + dt.timedelta(seconds=x))

In [ ]:
df['Data']

In [ ]:
normal = df[df['Class'] == 0].copy()
fraude = df[df['Class'] == 1].copy()

In [ ]:
print('Quantidade de registros normais: ', len(df[df['Class'] == 0]))
print('Quantidade de registros fraldados: ', len(df[df['Class'] == 1]))

Vamos visualizar a distribuição dos dados

In [ ]:
df.describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex = True)
plt.suptitle('Dispersão dos dados entre Valor da transação e Data da operação')

ax1.set_title('Normal')
sns.scatterplot(data=normal, x='Data', y='Amount', ax=ax1)

ax2.set_title('Fraude')
sns.scatterplot(data=fraude, x='Data', y='Amount', ax=ax2, color='red')

plt.xlabel('Data')
plt.ylabel('Valor da transação')
plt.tight_layout()
plt.show()

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex = True)
f.suptitle('Montante por transação por classe')

bins = 50

ax1.hist(fraude['Amount'], bins = bins)
ax1.set_title('Fraude')

ax2.hist(normal['Amount'], bins = bins)
ax2.set_title('Normal')

plt.xlabel('Total ($)')
plt.ylabel('Número de Transações')
plt.xlim((0, 20000))
plt.yscale('log')
plt.show()

In [ ]:
columns = df.columns[1:30]

plt.suptitle('Distribuição e Boxplot dos dados')
fig, axs = plt.subplots(nrows=len(columns), ncols=2, figsize=(16,90))

for i, column in enumerate(columns):
    ax0 = axs[i, 0]
    ax0.set_title('Histograma [{}]'.format(column))
    sns.histplot(data=df, x=column, hue='Class', ax=ax0)
    
    ax1 = axs[i, 1]
    ax1.set_title('Boxplot [{}]'.format(column))
    sns.boxplot(data=df, x=column, hue='Class', ax=ax1, orient='h')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.title('Correlação')

mask = np.zeros_like(df.corr(), dtype=np.bool8)
mask[np.triu_indices_from(mask)]= True


sns.heatmap(df.corr(), mask=mask, square=True, linewidths=0.5, cmap='viridis', vmin=-1, vmax=1, annot_kws={'size': 12})
plt.show()

Podemos notar que não existe uma correlação muito forte entre os dados, apenas uma feature ou outra possui um correlacionamento. Vamos remover os correlacionamentos com valor acima de 50%

In [ ]:
CORR = 0.5

In [ ]:
corr_matrix = df.drop(columns=['Class', 'Time', 'Data']).corr().abs()

sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool8))
                  .unstack()
                  .sort_values(ascending=False))
col_corr = list(map(lambda x: x[1], sol[sol.values > CORR].keys()))
print('Colunas com correlação > {} %'.format(CORR), col_corr)

## Modelagem dos dados


Vamos remover a informação de `Time, Class e Data` e as colunas com correlação do dataset pois esses features não serão utilizadas na previsão e vamos precisar padronizar o valor de `Amount` para que não haja diferença de escala com base nos outros valores das demais features

In [ ]:
scaler = StandardScaler()
scaler.fit(df['Amount'].values.reshape(-1, 1))

normal['Amount'] = scaler.transform(normal['Amount'].values.reshape(-1,1))
fraude['Amount'] = scaler.transform(fraude['Amount'].values.reshape(-1,1))

In [ ]:
y_normal = normal['Class'].values
y_fraude = fraude['Class'].values

In [ ]:
normal = normal.drop(columns=['Class', 'Time', 'Data']).drop(columns=col_corr)
fraude = fraude.drop(columns=['Class', 'Time', 'Data']).drop(columns=col_corr)

## Matriz de custo benefício

Vamos criar uma função para validar aqui o quanto de benefício nosso modelo trará ao identificar as fraudes e o custo que teremos caso essas fraudes passem.

### Definição para a validação do custo/benefício

!['Fraude com cartão'](./images/fraude.png)

https://risk.lexisnexis.com/global/pt/about-us/press-room/press-release/20211020-true-cost-of-fraud-latam

Dado que o valor de uma transação seja 1,00 real e que para uma empresa checar se o consumidor realmenente adquiriu um bem ela gaste 10% desse valor e 386% o valor da transação para solucionar a fraude, qual o impacto operacional se um modelo conseguir impedir uma fraude ?

In [ ]:
def calcular_custo_beneficio(tp, fp, tn, fn, valor_transacao = 1.0):
    """Exibe uma matriz de custo/benefício"""
 
    receita_operacional = tp * valor_transacao
    fraude_identificada = fp * valor_transacao * 3.86
    
    receita_validada = tn * valor_transacao * .9
    fraude_validada = tn * valor_transacao * .1
    
    receita_fraudada = fn * valor_transacao * 3.86
    
    print('Total de transações: {}'.format(tp + tn + fp + fn))
    print('-'.center(20, '-'))
    
    print('{} Transações válidas'.format(tp + tn))
    print('= Receita total: {:.2f}'.format(receita_operacional + receita_validada))
    print('+ {} Transações válidas: {:.2f}'.format(tp, receita_operacional))
    print('+ {} Transações checadas: {:.2f}'.format(tn, receita_validada))
    print('- Custom com checagem: {:.2f}'.format(fraude_validada))

    print('-'.center(20, '-'))
    print('{} Fraudes'.format(fp + fn))
    print('- Custo {:.2f}'.format(receita_fraudada))
    print('+ Fraude identificada (save): {:.2f}'.format(fraude_identificada))    

## Base line

Vamos criar um modelo baseline para o nosso modelo principal. O objetivo desse modelo base line e validar como duas estruturas de aprendizados podem chegar a um valor muito próximo um do outro.

Vamos utilizar um classificador do tipo Support Vector Machine para fazer a classificação dos dados. Para isso, vamos obter uma amostra do dataset de registros normais com a mesma quantidade de registros fraudados.

In [ ]:
X = normal.sample(len(fraude))
y = y_normal[X.index - 1]

X = pd.concat([X, fraude])
y = np.concatenate((y, y_fraude), axis=None)

Vamos criar as massas de treino e teste e criar o classificador SVM

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
svc = SVC(kernel='linear', random_state=RANDOM_SEED)
svc.fit(X_train, y_train)

Vamos verificar como foi o treinamento do modelo

In [ ]:
print('Score: {}' .format(svc.score(X_train, y_train)))

Vamos agora avaliar o teste

In [ ]:
y_test_pred = svc.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

Vamos validar se o modelo conseguiria identificar os registros de fraude na base desbalanceada

In [ ]:
X = pd.concat([normal, fraude])
y = np.concatenate((y_normal, y_fraude), axis=None)

In [ ]:
indexs = np.random.permutation(len(X))

X = X.iloc[indexs]
y = y[indexs]

In [ ]:
y_pred = svc.predict(X)

In [ ]:
print(confusion_matrix(y, y_pred))
print(classification_report(y, y_pred))

Vamos agora validar o quanto o nosso modelo performance em questão de custo/beneficio

In [ ]:
cf_svc = pd.DataFrame(confusion_matrix(y, y_pred), columns=LABELS)
cf_svc.index = LABELS
cf_svc

In [ ]:
calcular_custo_beneficio(cf_svc.loc['Normal']['Normal'], cf_svc.loc['Fraude']['Fraude'],
                           cf_svc.loc['Normal']['Fraude'], cf_svc.loc['Fraude']['Normal'])

## Arquitetura do modelo

Vamos utilizar um modelo baseado em Autoencoders, com aprendizado não-supervisioinado, para aprender as caracteristicas de transações normais e vamos utilizar do processo de reconstrução para identificar os registros com anomalias, uma vez que os registros com `Fraude` terão um erro de reconstrução maior devido ao modelo não "conhecer" essas caracteristicas que o definem como fraudulento.

!['Autoencoder'](./images/autoencoders.png)

Vamos dividir os dados em treino e teste para criar nosso modelo. Como vamos criar um modelo baseado em `autoencoders` e aprender as caracteristicas das transações válidas, vamos filtrar primeiramente apenas os registros com a classe `Normal` e dividi-los em sub-amostragens para treino e teste. Os registros com classe `Fraude` serão utilizadas apenas no processo de teste.

In [ ]:
train, test = train_test_split(normal, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
print('Treino: ', train.shape)
print('Test: ', test.shape)

In [ ]:
features = len(train.columns)
print(features)

### Definição do modelo

Vamos definir nosso modelo inicialmente criando a camada de encoder e posterior a camanda de decoder.

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
input_shape = (features,)
print(input_shape)

In [ ]:
encoder = Sequential([
    Dense(24, input_shape=input_shape, kernel_initializer='random_normal'),
    BatchNormalization(),
    LeakyReLU(),
    Dense(18, kernel_initializer='random_normal'),
    LeakyReLU(),
    Dense(12, kernel_initializer='random_normal'),
    LeakyReLU(),
    Dense(6, kernel_initializer='random_normal'),
    LeakyReLU(),
], name='encoding')

decoder = Sequential([
    Dense(12, kernel_initializer='random_normal'),
    BatchNormalization(),
    LeakyReLU(),
    Dense(18, kernel_initializer='random_normal'),
    LeakyReLU(),
    Dense(24, kernel_initializer='random_normal'),
    LeakyReLU(),
    Dense(train.shape[1], activation='linear', kernel_initializer='random_normal')
], name='decoding')

In [ ]:
model = Sequential([encoder, decoder], name='autoencoder')

In [ ]:
model.compile(optimizer = Adam(0.01), loss = 'mse', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
encoder.summary(), decoder.summary()

In [ ]:
rp = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=10, min_lr=0.001, factor=0.01)

In [ ]:
cp = ModelCheckpoint('./model/autoencoders.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

In [ ]:
with tf.device('/GPU:0'):
    history = model.fit(train, train, batch_size=256, validation_data=(test, test), epochs=500, callbacks=[cp, rp])

### Avaliando o treinamento do modelo

In [ ]:
plt.suptitle('Analise do modelo treinado')

plt.subplot(211)
plt.title('Erro do Modelo')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('Erro')
plt.xlabel('Epoch')
plt.legend(['Treino', 'Teste'], loc = 'best')

plt.subplot(212)
plt.title('Acurácia do Modelo')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('Acurácia')
plt.xlabel('Epoch')
plt.legend(['Treino', 'Teste'], loc = 'best')

plt.tight_layout()
plt.show()

### Carregando o modelo treinado

In [ ]:
model = tf.keras.models.load_model('./model/autoencoders.h5') 

### Executando o modelo para prever as classes de testes

Vamos inserir agora nos dados de testes os registros de fraude e validar como o modelo conseguirá classificar os registros de fraude

In [ ]:
test = pd.concat([test, fraude])
test = test.iloc[np.random.permutation(len(test))]

In [ ]:
test_pred = model.predict(test)

Vamos identificar agora o erro da reconstrução gerada pelo modelo. O erro de reconstrução vai ser utilizado para prever a probabilidade de uma amostra ser uma instância fraudada. 

O fato de termos utilizado apenas registros `normais` para o treinamento do modelo do autoencoder é que durante o processo de inferência os registros com fraude perde as caracteristicas da fraude e o decodificador as reconstroi como sendo um registro normal, resultando em um grande erro de reconstrução.

Depois de calcular todos os erros no dataset de teste podemos gerar uma probabilidade, entre 0 e 1, indicando o percentual de anômalia que essa instancia possui. Com base nisso, podemos definir um threshold para limitar quais registros são anômalos.

### Função de calculo de erro para reconstrução

Vamos buscar as labels para os registros de teste

In [ ]:
labels = df.iloc[test.index]['Class'].values

Vamos utilizar o erro quadrático médio para achar o erro de reconstrução do modelo.

$$\textstyle L(x,x') = ||\, x - x'||^2$$

In [ ]:
labels_pred = np.mean(np.power(test-test_pred, 2), axis=1)

In [ ]:
labels_pred = np.array(labels_pred).reshape(-1,1)

In [ ]:
labels_pred[0:5]

Vamos normalizar os erros para eles ficarem em uma mesma escala

In [ ]:
erro_scaler = MinMaxScaler()
previsao_fraude = erro_scaler.fit_transform(labels_pred).flatten()

In [ ]:
previsao_fraude[0:5]

In [ ]:
true_labels = [i for i, label in enumerate(labels) if label == 1]

print(labels[true_labels[0:5]])
print(previsao_fraude[true_labels[0:5]])

Vamos verificar agora se o modelo foi capaz de identificar as anomalias nas transações.

In [ ]:
plt.figure(figsize=(20,10))
plt.title('Transações com % de probabilidade de ser fraude')
plt.plot(labels, c='blue', label='Transação')
plt.plot(previsao_fraude, c='red', label='$\hat{P}$ fraude')
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('Transações')
plt.ylabel('Score de fraudes')
plt.legend()
plt.show()

## Avaliação do modelo

Vamos avaliar agora se o nosso modelo teve uma boa taxa de acerto nos dados de testes. Vamos utilizar a curva ROC-AUC para medir a eficácia de nosso modelo em distingir as duas classes. 

Valores perto do canto superior esquerdo, perto de 1, indicam que o classificar é bom em distinguir as classes e valores abaixo da área média, perto de 0.5, indicam que o modelo não conseguiu distinguir entre as classes.

In [ ]:
fpr, tpr, threshold = roc_curve(labels, previsao_fraude)

In [ ]:
score_auc = roc_auc_score(labels, previsao_fraude)
print('Score AUC: ', score_auc)

In [ ]:
plt.figure(figsize = (10,5))
plt.title('ROC')
plt.plot([0, 1], [0, 1], color = 'black', linestyle = '--')
plt.plot(fpr, tpr, label = 'AUC = {}'.format(score_auc))
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.show()

Nosso modelo conseguiu classificar bem os eventos com transações com as duas labels. Porém como temos dados desbalanceados o ROC pode não expressar corretamente o quao bom ou ruim esta nosso modelo. Vamos utilizar para isso outras métricas de avaliação.

## Precisão vs Recal

!['Precisao vs Recall'](./images/precision_recall.png)

Precisão e Recall são métricas para avaliar o quanto um modelo esta conseguindo identificar as classes corretas durante um processo de classificação. Porém a precisão e o recall tem objetivos distintos na identificação do quão bem o modelo esta classificando.

Precisão e recall são definidos da seguinte forma:

$$\text{Precision} = \frac{\text{true positives}}{\text{true positives} + \text{false positives}}$$

$$\text{Recall} = \frac{\text{true positives}}{\text{true positives} + \text{false negatives}}$$

* A precisão mede a relevância dos resultados obtidos. 

* Recall, por outro lado, mede quantos resultados relevantes são retornados.

Ambos os valores podem ter valores entre 0 e 1.

Vamos calcular agora os valores e plotar em um gráfico

In [ ]:
precision, recall, th = precision_recall_curve(labels, previsao_fraude)

In [ ]:
plt.title('Recall vs Precisão')
plt.plot(recall, precision, 'b', label = 'Curva Precisão-Recall')
plt.xlabel('Recall')
plt.ylabel('Precisão')
plt.show()

In [ ]:
plt.plot(th, precision[1:], 'b', label = 'Curva Threshold-Precisão')
plt.title('Precisão Para Diferentes Valores de threshold')
plt.xlabel('Threshold')
plt.ylabel('Precisão')
plt.show()

In [ ]:
plt.plot(th, recall[1:], 'b', label = 'Curva Threshold-Recall')
plt.title('Recall Para Diferentes Valores de threshold')
plt.xlabel('Threshold')
plt.ylabel('Recall')
plt.show()

## Identificando fraudes

Precisamos agora definir um limite de threshold para o nosso modelo. Valores acima desse limite indicaram que os registros possuem fraude na transação, então precisamos achar um valor que consiga identificar o maior número possível de fraudes.

Vamos utilizar como base a métrica de F1-Score, que é uma média harmônica de precisão e recall.

In [ ]:
limites_fraude = [(previsao_fraude > i).astype(np.int32) for i in threshold]
f1_scores = [f1_score(labels, i) for i in limites_fraude]

In [ ]:
plt.figure(figsize = (10, 5))
plt.title('F-1 Score vs Thresholds')
plt.plot(threshold, f1_scores)
plt.xlabel('Thresholds')
plt.ylabel('F-1 Score')
plt.show()

Vamos obter o melhor threshold

In [ ]:
print(np.min(f1_scores))
print(np.max(f1_scores))

In [ ]:
threshold

In [ ]:
melhor_threshold = threshold[f1_scores.index(np.max(f1_scores))]
print('Melhor Threshold = {}'.format(melhor_threshold))

Vamos agora classificar nossos registros com base nesse threshold

In [ ]:
indicador_fraude = (previsao_fraude > (melhor_threshold)).astype(np.int32)

In [ ]:
indicador_fraude

Vamos analisar agora como ficou as classificações dos registros com base no threshold selecionado

In [ ]:
cf_model = pd.DataFrame(confusion_matrix(labels, indicador_fraude), columns=LABELS)
cf_model.index = LABELS
print(cf_model)

In [ ]:
indicador_fraude_final = ['normal' if i == 0 else 'fraude' for i in indicador_fraude]

In [ ]:
plt.figure(figsize=(20,10))
plt.title('Transações com % de probabilidade de ser fraude')
plt.plot(labels, c='blue', label='Transação')
plt.plot(previsao_fraude, c='red', label='$\hat{P}$ fraude')

plt.axhline(y = melhor_threshold, linestyle = '--', label = 'threshold', color='black')

plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('Transações')
plt.ylabel('Score de fraudes')
plt.legend()
plt.show()

## Metricas de performance

In [ ]:
precision = precision_score(labels, indicador_fraude)
recall = recall_score(labels, indicador_fraude)
f1_sc = f1_score(labels, indicador_fraude)
accuracy_sc = accuracy_score(labels, indicador_fraude)

In [ ]:
print("""Métricas de Avaliação do Modelo:
         Precision = {}
         Recall = {}
         Score F1 = {}
         Acurácia = {}"""
      .format(precision, recall, f1_sc, accuracy_sc))

In [ ]:
print(classification_report(labels, indicador_fraude))

In [ ]:
calcular_custo_beneficio(cf_model.loc['Normal']['Normal'], cf_model.loc['Fraude']['Fraude'],
                           cf_model.loc['Normal']['Fraude'], cf_model.loc['Fraude']['Normal'])